# Ad-hoc запросы к базе данных сервиса чтения книг

Выполнила Наталья Яковлева, когорта 67.

<b> Контекст </b> коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

<b> Задача: </b> проанализировать базу данных.

<b> Описание данных: </b>
- Таблица `books` содержит данные о книгах; 
- Таблица `authors` содержит данные об авторах; 
- Таблица `publishers` содержит данные об издательствах;
- Таблица `ratings` содержит данные о пользовательских оценках книг;
- Таблица `reviews` содержит данные о пользовательских обзорах на книги.

<b> Задания: </b> 
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<b> Ход проекта: </b>
1. [Чтение данных](#reading)
2. [Запрос 1](#query1)
3. [Запрос 2](#query2)
4. [Запрос 3](#query3)
5. [Запрос 4](#query4)
6. [Запрос 5](#query5)
7. [Общий вывод](#ending)

In [23]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
# db_config = {'user': '', # имя пользователя
# 'pwd': '', # пароль
# 'host': '',
# 'port': , # порт подключения
# 'db': ''} # название базы данных
#connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
#engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Загрузка таблиц <a name="reading"></a>

### Таблица `books`

In [2]:
query = '''SELECT * FROM books LIMIT 5'''
books = pd.io.sql.read_sql(query, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица `authors`

In [3]:
query = '''SELECT * FROM authors LIMIT 5'''
authors = pd.io.sql.read_sql(query, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица `publishers`

In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица `ratings`

In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица `reviews`

In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<b> Вывод: </b> Успешно загрузила хэды пяти таблиц, далее напиши нужные запросы.

## Написание запросов

### Сколько книг вышло после 1 января 2000 года? <a name="query1"></a>

In [8]:
query = '''
    SELECT COUNT(book_id) books_count
    FROM books
    WHERE publication_date > '2000-01-01'
'''
books_count_from_01_01_2000 = pd.io.sql.read_sql(query, con = engine)
books_count_from_01_01_2000

,books_count
0,819


<b> Вывод: </b> после 1 января 2000г. было опубликовано 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку <a name="query2"></a>

In [9]:
query = '''
    WITH reviews AS (
        SELECT book_id, COUNT(DISTINCT review_id) review_count
        FROM reviews 
        GROUP BY book_id),
        
    rating AS (
        SELECT book_id, AVG(rating) avg_rating
        FROM ratings
        GROUP BY book_id)
                   
    SELECT reviews.book_id, books.title, rating.avg_rating, reviews.review_count
    FROM reviews
    JOIN rating ON rating.book_id = reviews.book_id
    JOIN books ON books.book_id = reviews.book_id
'''
books_reviews_and_avg_rating = pd.io.sql.read_sql(query, con = engine)
books_reviews_and_avg_rating

,book_id,title,avg_rating,review_count
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
989,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
990,997,Xenocide (Ender's Saga #3),3.400000,3
991,998,Year of Wonders,3.200000,4
992,999,You Suck (A Love Story #2),4.500000,2


In [10]:
books_reviews_and_avg_rating.isna().sum()

book_id         0
title           0
avg_rating      0
review_count    0
dtype: int64

In [11]:
books_reviews_and_avg_rating.query('avg_rating == 5.00')['book_id'].count()

43

In [12]:
books_reviews_and_avg_rating['review_count'].max()

7

In [13]:
books_reviews_and_avg_rating.query('review_count == 7')

,book_id,title,avg_rating,review_count
941,948,Twilight (Twilight #1),3.6625,7


In [14]:
books_reviews_and_avg_rating['review_count'].min()

1

In [15]:
books_reviews_and_avg_rating.query('review_count == 1')

,book_id,title,avg_rating,review_count
1,2,1 000 Places to See Before You Die,2.500000,1
13,14,A Christmas Carol and Other Christmas Writings,4.500000,1
90,92,Babyville,3.500000,1
127,129,Built to Last: Successful Habits of Visionary ...,4.500000,1
154,156,Color: A Natural History of the Palette,4.333333,1
163,165,Creepshow,4.500000,1
184,186,Death: The High Cost of Living,3.000000,1
185,187,Debt of Honor (Jack Ryan #7),3.000000,1
193,196,Dragons of Winter Night (Dragonlance: Chronicl...,4.000000,1
222,226,Everyday Italian: 125 Simple and Delicious Rec...,3.500000,1


In [16]:
books_reviews_and_avg_rating.query('review_count == 1').count()

book_id         37
title           37
avg_rating      37
review_count    37
dtype: int64

<b> Вывод: </b> расчитала для каждой книги количество обзоров и средний рейтинг (получила 994 строки). Максимальный средний рейтинг в 5 баллов получили 7 книг. Больше всего отзывов оставлено на книгу "Сумерки" - 7 обзоров со средней оценкой 3.66. На 37 книг было оставлено по 1 отзыву.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц <a name="query3"></a>

In [17]:
query = '''
    SELECT pub.publisher_id, pub.publisher, COUNT(books.book_id) books_count
    FROM publishers pub
    JOIN books ON pub.publisher_id = books.publisher_id
    WHERE books.num_pages > 50
    GROUP BY pub.publisher_id
    ORDER BY books_count DESC
    LIMIT 1
'''
the_publisher = pd.io.sql.read_sql(query, con = engine)
the_publisher

,publisher_id,publisher,books_count
0,212,Penguin Books,42


<b> Вывод: </b> издательство Penguin Books выпустило наибольше кол-во книг толще 50 стр. - 42 книги.

### Определите автора с самой высокой средней оценкой книг <a name="query4"></a>

Необходимо учитывать только книги с 50 и более оценками:

In [18]:
query = '''
    WITH popular_books AS (
        SELECT book_id, AVG(rating) avg_rating, COUNT(book_id) reviews_count
        FROM ratings
        GROUP BY book_id
        ORDER BY reviews_count DESC
    )
    
    SELECT authors.author, AVG(popular_books.avg_rating) avg_rating 
    FROM books 
    LEFT OUTER JOIN authors ON books.author_id = authors.author_id
    LEFT OUTER JOIN popular_books ON popular_books.book_id = books.book_id
    WHERE reviews_count >= 50
    GROUP BY authors.author
    ORDER BY avg_rating DESC
    LIMIT 1
'''
the_author = pd.io.sql.read_sql(query, con = engine)
the_author

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


<b> Вывод: </b> среди книг с 50 и более оценок автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок <a name="query5"></a>

In [19]:
query = '''
    WITH users AS (
        SELECT username, COUNT(rating_id) ratings_count
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48),
        
    reviewers AS (
        SELECT username, COUNT(review_id) reviews_count
        FROM reviews
        GROUP BY username)

    SELECT SUM(reviewers.reviews_count) / COUNT(users.username) avg_review_count
    FROM users 
    JOIN reviewers ON users.username = reviewers.username
'''
a = pd.io.sql.read_sql(query, con = engine)
a

,avg_review_count
0,24.0


<b> Вывод: </b> пользователи, которые поставили больше 48 оценок, в среднем делают 24 обзора.

## Общий вывод: <a name="ending"></a>

Исследовала базу данных сервиса для чтения книг по подписке. Можно выделить следующее:

1. После 1 января 2000г. было опубликовано 819 книг.
2. Расчитала для каждой книги количество обзоров и средний рейтинг. Максимальный средний рейтинг в 5 баллов получили 7 книг. Больше всего отзывов оставлено на книгу "Сумерки" - 7 обзоров со средней оценкой 3.66. На 37 книг было оставлено по 1 отзыву.
3. Издательство Penguin Books выпустило наибольшее кол-во книг толще 50 стр. - 42.
4. Среди книг с 50 и более оценок автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.
5. Пользователи, которые поставили больше 48 оценок, в среднем делают 24 обзора.